# Pothole Detection System

This notebook demonstrates automatic pothole detection from road images using computer vision techniques.

## Approach
1. **Preprocessing**: Grayscale conversion, noise reduction, contrast enhancement
2. **Edge Detection**: Canny edge detector to find boundaries
3. **Morphological Operations**: Close gaps in edge segments
4. **Contour Analysis**: Find and filter shapes based on geometric properties
5. **Result Visualization**: Draw bounding boxes and count potholes

In [ ]:
# Import required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import os

# Set up matplotlib for better display
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['axes.titlesize'] = 14

In [ ]:
def display_images(images, titles, figsize=(15, 5)):
    """Helper function to display multiple images"""
    fig, axes = plt.subplots(1, len(images), figsize=figsize)
    if len(images) == 1:
        axes = [axes]
    
    for i, (img, title) in enumerate(zip(images, titles)):
        if len(img.shape) == 3:
            axes[i].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        else:
            axes[i].imshow(img, cmap='gray')
        axes[i].set_title(title)
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

In [ ]:
def preprocess_image(image):
    """Preprocess the image for better pothole detection"""
    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(blurred)
    
    return gray, blurred, enhanced

In [ ]:
def detect_potholes_step_by_step(image_path):
    """Detect potholes with step-by-step visualization"""
    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load image {image_path}")
        return
    
    print(f"Processing: {os.path.basename(image_path)}")
    
    # Step 1: Preprocessing
    gray, blurred, enhanced = preprocess_image(image)
    
    print("\n1. Preprocessing Steps:")
    display_images([image, gray, blurred, enhanced], 
                  ['Original', 'Grayscale', 'Blurred', 'Enhanced'], 
                  figsize=(20, 5))
    
    # Step 2: Edge detection
    edges = cv2.Canny(enhanced, 50, 150)
    
    # Step 3: Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    closed = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)
    
    print("\n2. Edge Detection and Morphological Operations:")
    display_images([edges, closed], ['Canny Edges', 'After Closing'], figsize=(12, 5))
    
    # Step 4: Find contours
    contours, _ = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw all contours for visualization
    all_contours_img = image.copy()
    cv2.drawContours(all_contours_img, contours, -1, (0, 255, 0), 2)
    
    # Step 5: Filter contours
    height, width = image.shape[:2]
    potholes = []
    min_area = 200
    max_area = width * height * 0.3
    
    for contour in contours:
        area = cv2.contourArea(contour)
        
        if min_area < area < max_area:
            perimeter = cv2.arcLength(contour, True)
            if perimeter > 0:
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = float(w) / h
                
                if (0.1 < circularity < 1.2 and 
                    0.3 < aspect_ratio < 3.0 and
                    w > 20 and h > 20):
                    
                    potholes.append({
                        'contour': contour,
                        'area': area,
                        'bbox': (x, y, w, h),
                        'circularity': circularity
                    })
    
    print("\n3. Contour Detection:")
    display_images([all_contours_img], [f'All Contours ({len(contours)} found)'], figsize=(10, 6))
    
    # Step 6: Draw final results
    result_image = image.copy()
    
    for i, pothole in enumerate(potholes):
        # Draw contour
        cv2.drawContours(result_image, [pothole['contour']], -1, (0, 255, 0), 3)
        
        # Draw bounding box
        x, y, w, h = pothole['bbox']
        cv2.rectangle(result_image, (x, y), (x + w, y + h), (255, 0, 0), 3)
        
        # Add label
        label = f"Pothole {i+1}"
        cv2.putText(result_image, label, (x, y-10), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
    
    # Add summary
    summary_text = f"Potholes detected: {len(potholes)}"
    cv2.putText(result_image, summary_text, (10, 40), 
               cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
    
    print("\n4. Final Results:")
    display_images([image, result_image], ['Original Image', f'Detected Potholes: {len(potholes)}'], figsize=(16, 8))
    
    # Print detailed results
    print(f"\n📊 DETECTION SUMMARY:")
    print(f"Total potholes detected: {len(potholes)}")
    
    for i, pothole in enumerate(potholes):
        area_sqm = pothole['area'] / 10000  # Rough conversion
        print(f"  Pothole {i+1}: Area = {pothole['area']:.0f} pixels (~{area_sqm:.2f} sq.m), Circularity = {pothole['circularity']:.2f}")
    
    return len(potholes), result_image

## Load and Process Images

Place your road images in the `input_images` folder, then run the cell below to process them:

In [ ]:
# Find all images in the input folder
input_folder = "input_images"
extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']

image_files = []
for ext in extensions:
    image_files.extend(Path(input_folder).glob(f'*{ext}'))
    image_files.extend(Path(input_folder).glob(f'*{ext.upper()}'))

print(f"Found {len(image_files)} image(s) to process")
for img_file in image_files:
    print(f"  - {img_file.name}")

In [ ]:
# Process each image with detailed visualization
total_potholes = 0
results = []

for image_path in image_files:
    print("\n" + "="*80)
    count, result_img = detect_potholes_step_by_step(str(image_path))
    total_potholes += count
    results.append((image_path.name, count, result_img))
    print("="*80)

print(f"\n🎯 FINAL SUMMARY:")
print(f"Total images processed: {len(image_files)}")
print(f"Total potholes detected: {total_potholes}")

## Save Results

Save the processed images with detected potholes:

In [ ]:
# Create output directory and save results
output_folder = Path("output_results")
output_folder.mkdir(exist_ok=True)

for filename, count, result_img in results:
    output_path = output_folder / f"detected_{filename}"
    cv2.imwrite(str(output_path), result_img)
    print(f"Saved: {output_path}")

print(f"\n✅ All results saved in '{output_folder}' folder!")